In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv(r"temperature_data.csv")
df.drop(columns = [df.columns[0]], inplace=True)

In [3]:
df

,date,Min Temp,Max Temp,Mean Temp
0,1951-01-01,2.680000,15.130000,8.905000
1,1951-01-02,3.030000,18.340000,10.685000
2,1951-01-03,2.910000,17.370001,10.140000
3,1951-01-04,3.170000,17.760000,10.465000
4,1951-01-05,3.650000,16.490000,10.070000
...,...,...,...,...
26658,2023-12-27,7.624851,18.110460,12.867656
26659,2023-12-28,7.640046,15.210753,11.425400
26660,2023-12-29,7.504602,15.522985,11.513794
26661,2023-12-30,7.527018,14.187593,10.857306


In [4]:
df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.to_period('M')
monthly_data = df.drop(columns=['date'])
monthly_data = monthly_data.groupby('year_month').mean()
df.drop(columns=['year_month'], inplace=True)
print(monthly_data)

             Min Temp   Max Temp  Mean Temp
year_month                                 
1951-01      3.967419  15.801935   9.884677
1951-02      5.838214  19.208929  12.523571
1951-03     10.764839  23.542581  17.153710
1951-04     13.592667  26.587000  20.089833
1951-05     19.187097  32.264839  25.725968
...               ...        ...        ...
2023-08     21.749655  28.558658  25.154156
2023-09     20.565490  28.811419  24.688454
2023-10     14.921647  26.846221  20.883934
2023-11     10.921349  22.451928  16.686639
2023-12      7.025675  18.559081  12.792378

[876 rows x 3 columns]


In [5]:
df['year'] = df['date'].dt.to_period('Y')
yearly_data = df.drop(columns=['date'])
yearly_data = yearly_data.groupby('year').mean()
df.drop(columns=['year'], inplace=True)
print(yearly_data)

       Min Temp   Max Temp  Mean Temp
year                                 
1951  14.310137  25.828137  20.069137
1952  14.381284  25.828388  20.104836
1953  14.853123  25.930548  20.391836
1954  14.385726  25.159151  19.772438
1955  14.112247  25.138329  19.625288
...         ...        ...        ...
2019  14.263684  24.403127  19.333405
2020  14.057682  24.611245  19.334464
2021  14.558620  25.328709  19.943665
2022  15.151315  25.707047  20.429181
2023  14.523701  24.966711  19.745206

[73 rows x 3 columns]


In [10]:
df

,date,Min Temp,Max Temp,Mean Temp
0,1951-01-01,2.680000,15.130000,8.905000
1,1951-01-02,3.030000,18.340000,10.685000
2,1951-01-03,2.910000,17.370001,10.140000
3,1951-01-04,3.170000,17.760000,10.465000
4,1951-01-05,3.650000,16.490000,10.070000
...,...,...,...,...
26658,2023-12-27,7.624851,18.110460,12.867656
26659,2023-12-28,7.640046,15.210753,11.425400
26660,2023-12-29,7.504602,15.522985,11.513794
26661,2023-12-30,7.527018,14.187593,10.857306


In [7]:
df_rain = pd.read_csv(r"rainfall_data.csv")
df_rain.drop(columns = [df_rain.columns[0]], inplace=True)

In [8]:
df_rain

,LONGITUDE,LATITUDE,TIME,RAINFALL
0,78.5,30.5,1951-01-01,0.0
1,78.5,30.5,1951-01-02,0.0
2,78.5,30.5,1951-01-03,0.0
3,78.5,30.5,1951-01-04,0.0
4,78.5,30.5,1951-01-05,0.0
...,...,...,...,...
26293,78.5,30.5,2022-12-27,0.0
26294,78.5,30.5,2022-12-28,0.0
26295,78.5,30.5,2022-12-29,0.0
26296,78.5,30.5,2022-12-30,0.0


In [9]:
df_rain['TIME'] = pd.to_datetime(df_rain['TIME'])
df_rain['year_month'] = df_rain['TIME'].dt.to_period('M')
monthly_rain_data = df_rain.drop(columns=['TIME'])
monthly_rain_data = monthly_rain_data.groupby('year_month').mean()
df_rain.drop(columns=['year_month'], inplace=True)
print(monthly_rain_data)

            LONGITUDE  LATITUDE   RAINFALL
year_month                                
1951-01          78.5      30.5   1.495463
1951-02          78.5      30.5   0.647581
1951-03          78.5      30.5   2.439290
1951-04          78.5      30.5   0.139405
1951-05          78.5      30.5   1.566789
...               ...       ...        ...
2022-08          78.5      30.5  13.266669
2022-09          78.5      30.5  11.473897
2022-10          78.5      30.5   3.875470
2022-11          78.5      30.5   0.000000
2022-12          78.5      30.5   0.000000

[864 rows x 3 columns]


In [11]:
df_rain['year'] = df_rain['TIME'].dt.to_period('Y')
yearly_rain_data = df_rain.drop(columns=['TIME'])
yearly_rain_data = yearly_rain_data.groupby('year').mean()
df_rain.drop(columns=['year'], inplace=True)
print(yearly_rain_data)

      LONGITUDE  LATITUDE  RAINFALL
year                               
1951       78.5      30.5  4.010587
1952       78.5      30.5  4.623981
1953       78.5      30.5  4.778490
1954       78.5      30.5  5.834388
1955       78.5      30.5  5.474035
...         ...       ...       ...
2018       78.5      30.5  5.519811
2019       78.5      30.5  4.108388
2020       78.5      30.5  1.116805
2021       78.5      30.5  5.285882
2022       78.5      30.5  4.746849

[72 rows x 3 columns]


In [12]:
df_rain

,LONGITUDE,LATITUDE,TIME,RAINFALL
0,78.5,30.5,1951-01-01,0.0
1,78.5,30.5,1951-01-02,0.0
2,78.5,30.5,1951-01-03,0.0
3,78.5,30.5,1951-01-04,0.0
4,78.5,30.5,1951-01-05,0.0
...,...,...,...,...
26293,78.5,30.5,2022-12-27,0.0
26294,78.5,30.5,2022-12-28,0.0
26295,78.5,30.5,2022-12-29,0.0
26296,78.5,30.5,2022-12-30,0.0
